In [1]:
import geopandas as gpd
from datetime import timedelta
from IPython.display import display
from shapely.geometry import Polygon
from utils import process_and_save_data, load_processed_data
from model import RobustLandUsePredictionModel
from views import create_band_viewer, create_classification_viewer, create_spectral_indices_viewer, create_map_with_slider, analyze_time_series, plot_time_series, create_prediction_map_with_slider


In [ ]:
roi_coords = [
    [75.58679011123377, 26.725580815770467],
    [76.00152399795252, 26.725580815770467],
    [76.00152399795252, 27.08929110630265],
    [75.58679011123377, 27.08929110630265],
    [75.58679011123377, 26.725580815770467]
]

roi_polygon = Polygon(roi_coords)
roi_gdf = gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[roi_polygon])



In [ ]:
# Define file paths
data_dir = 'DataSet'
processed_dir = 'Processed_DataSet'
bounds = [[26.725580815770467, 75.58679011123377], 
         [27.08929110630265, 76.00152399795252]]

# Analyze time series
results = analyze_time_series(data_dir)

# Plot the time series results
plot_time_series(results)


In [ ]:
all_processed_data = process_and_save_data(data_dir, processed_dir)


In [ ]:
band_viewer = create_band_viewer('DataSet')
display(band_viewer)


In [ ]:
spectral_viewer = create_spectral_indices_viewer(all_processed_data)
display(spectral_viewer)


In [ ]:
classification_viewer = create_classification_viewer(all_processed_data)
display(classification_viewer)


In [ ]:
Map = create_map_with_slider(all_processed_data, bounds, roi_gdf)
Map


In [ ]:
'''
model = RobustLandUsePredictionModel(
        spatial_smoothing=1.0,
        chunk_size=100,
        n_estimators=50
    )
    
model.fit('Processed_DataSet')

model.save_model('land_use_model.joblib')
'''

In [ ]:
model = RobustLandUsePredictionModel.load_model('land_use_model.joblib')


In [ ]:
all_historical_data = load_processed_data(processed_dir)


In [ ]:
data_for_visualization = []

In [ ]:
if all_historical_data:
    all_historical_data.sort(key=lambda x: x['date'])
    current_date = all_historical_data[-1]['date']
    one_year_ago = current_date - timedelta(days=365)
    
    # Filter to get only the last 12 months of data
    recent_data = [
        data for data in all_historical_data 
        if data['date'] >= one_year_ago
    ]
    
    print(f"Using historical data from {recent_data[0]['date_str']} to {recent_data[-1]['date_str']}")
    print(f"Number of historical data points: {len(recent_data)}")
    
    # 3. Update model's historical data with recent data
    model.historical_data = recent_data
    
    # 4. Make predictions for future months
    try:
        predictions = model.predict_multiple_steps(num_steps=3)
        print("\nPredictions generated for:")
        for pred in predictions:
            print(f"- {pred['date_str']}")
        
        # 5. Combine recent historical and prediction data for visualization
        data_for_visualization = recent_data + predictions
        
        # 7. Analyze and print prediction summary
        print("\nPrediction Summary:")
        for i, pred in enumerate(predictions, 1):
            total_pixels = pred['classification'].size
            urban_pixels = (pred['classification'] == 3).sum()
            veg_pixels = ((pred['classification'] == 1) | 
                         (pred['classification'] == 2)).sum()
            
            urban_percent = (urban_pixels / total_pixels) * 100
            veg_percent = (veg_pixels / total_pixels) * 100
            
            print(f"\nMonth {i} ({pred['date_str']}):")
            print(f"- Urban coverage: {urban_percent:.1f}%")
            print(f"- Vegetation coverage: {veg_percent:.1f}%")
            print(f"- Average NDVI: {pred['NDVI'].mean():.3f}")
            print(f"- Average NDBI: {pred['NDBI'].mean():.3f}")
    
    except Exception as e:
        print(f"Error during prediction: {str(e)}")
else:
    print("No historical data found in the processed directory.")
    

In [ ]:

P_Map = create_prediction_map_with_slider(data_for_visualization, bounds, roi_gdf)
P_Map


In [ ]:
importance_df = model.get_feature_importance()


In [ ]:
print("\nFeature Importance:")
print(importance_df)


In [ ]:
summary = model.get_model_summary()
print("\nModel Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")
